<a href="https://colab.research.google.com/github/e-white25/Working-with-CRIPSR-data-QC-BWA-SAMtools-BCFtools/blob/main/Copy_of_E_White_CRISPR_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#CRIPSPR Sample data
!gdown 1-96T1PZKA_FQeD_ZK5z9USaHLP3jVdRO
!gdown 1-BXGr3XVGtd9Tx6PCHSp4hepK41GGCTS

#indexed mouse genome, mm10.
!gdown 1a8CP4P5zkzIBiw1EleqJiSwDW0VZcAar

In [ ]:
!tar -xf MM10.tar
!gzip -df MM10.tar.gz
# this command extracts the tar file to the current working folder/directory
# the [!gzip -df] command is compressing the collection of mouse genome data files; the .tar.gz indicates that the compressed "MM10" file is a tar file, i.e it contains multiple files, likely per individual chromosome or region of the genome

gzip: MM10.tar.gz: No such file or directory


In [ ]:
!apt install bwa
!apt install samtools
!apt install bcftools

# The mouse genome was already indexed for us; had we been given the raw file, BWA first requires an FM-index is created for the reference genome (via the index command)
# Had we not been given an indexed file, the command is: !bwa index -p MouseRef MM10/Mouse.fasta
# The index file had also already undergone the Burrows-Wheeler Transformation, allowing for an efficiently, and reversably compressed reference genome file
# BWA (Burrows-Wheeler Alignment Tool) uses a matrix algorithm to align CRISPR input reads to the mouse reference genome
# SAMtools is a software package for high-throughput sequencing data analysis
# BCFtools are used to manipulate variant call files (VCF) and binary call files (BCF); BCF tools can be used for compressed and uncompressed files.

In [ ]:
!sudo apt-get install -y default-jre
!wget https://www.bioinformatics.babraham.ac.uk/projects/fastqc/fastqc_v0.11.9.zip
!unzip fastqc_v0.11.9.zip
!chmod +x FastQC/fastqc
!pip install cutadapt

# sudo is a library required to run SAMtools
# chmod command stands for "change mode" to change the permissions of files and directories.
# +x 'executes' permission to the specified file or directory: FastQC/fastqc: This is the path to the file/directory we are changing permissions for
# cutadapt is a bioinformatics tool for processing high-throughput sequencing data; it will be used to remove low QC data and unwanted sequences, i.e. seq. data with primer interference

In [ ]:
!cutadapt --length -65 -o trimmed_CRISPR.R1.fastq -p trimmed_CRISPR.R2.fastq CRISPR.R1.fastq CRISPR.R2.fastq
#trimming off the low QC data due to junk reads from primers

In [ ]:
!bwa aln MM10/Mouse trimmed_CRISPR.R1fastq > CRISPR.R1.sai
!bwa aln MM10/Mouse trimmed_CRISPR.R2fastq > CRISPR.R2.sai

# bwa calls on the alighment tools; aln directs the algorithm to use short reads
# the first file calls the mouse reference genome/sequences that the CRISPR the reads are being aligned against
# the CRISPR input file contains the trimmed sequence reads in FASTQ format
# >CRISPR.R1/2.sai: Redirects the output of the alignment process to files named "CRISPR.R1/2.sai."
# .sai is a file that is formatted in a binary array; this allows for subsequent steps of the BWA alignment process

In [ ]:
!bwa sampe MM10/Mouse CRISPR.R1.sai CRISPR.R2.sai trimmed_CRISPR.R1.fastq trimmed_CRISPR.R2.fastq > CRISPR.bam

# The .sai files are the paired reference genome arrays; the fastq files are the sample files being aligned for analysis
# .fastq specifies the file with the BWA algorithm for paired-end reads (this just means the data set contains reads from both DNA strands; i.e primer pairs)
#> CRISPR.bam redirects the output to a BAM file

In [ ]:
!samtools view -q 12 CRISPR_Variants.bam | samtools sort -o CRISPR_Variants.bam.sorted | bcftools mpileup -f MM10.fasta --max-depth 2000 CRISPR_Variants.bam.sorted | bcftools call --multiallelic-caller --variants-only --ploidy 2 -Oz -o CRISPR.Variants.vcf.gz

# running SAM tools as a pipeline reduces the amount of RAM spaced used
# view: -q calls for the graph to read only the reads with the specified mapping quality; removing the ones with quality below 12
# sort -o: calls for the sorting of the previous 'cell's' BAM/SAM file; the bam.sorted file name is the output
# mpileup -f: summarizes all of the reads and the max-depth cuts off at 2000 bases per site
# mouse ploidy: diploid (2); multiallelic allows for multiallelic sites
#  --variants-only : Outputs varianst only
# -Oz: output compressed file format


In [ ]:
!gzip -df CRISPR.Variants.vcf.gz
# unzipping the compressed file

Parsing Variant Files

In [ ]:
!pip install cyvcf2
import cyvcf2
vcf_reader = cyvcf2.VCF('CRISPR.Variants.vcf')

with open('output.tsv', 'w') as file:

    for variant in vcf_reader:
        genotype = variant.gt_types[0]
        data_string = f"{variant.CHROM}\t{variant.POS}\t{variant.ID}\t{variant.REF}\t{','.join(variant.ALT)}\t{variant.QUAL}\t{variant.FILTER}\t{genotype}\n"
        file.write(data_string)

# cyvcf2 is library allowing for the parsing of VCF files; the library is required to read a VCF (Variant Call Format) files
# the output is where information will be extracted and stored to
# the array being created is called vcf_reader
# for each element (variant) in the array; the For loop will print each of the attributes listed in the string
# genotype = variant.gt_types[0];calls to return an array representing the genotypes for each sample
# the data string creates a string for each attribute being called for each varian
# the file.write calls for storing the data in a TSV (Tab-Separated Values) file

In [ ]:
#(1) A data frame with the data corresponding to the intended mutations
import pandas as pd
columns = ['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'GENOTYPE']
df = pd.read_csv("output.tsv", sep = "\t", header=None, names= columns)

chr2 = df[df['CHROM'] == 'chr2']

Edit_sites = [36937210, 36996899, 85400441, 85776687, 85918029, 86198668, 86236802, 86658391, 87049235]
Edit_sites_df = chr2[chr2['POS'].isin(Edit_sites)]
Edit_sites_df

# The pandas library is being imported to read the TSV file ("output.tsv"); and to sort the data into a DataFrame with the specified column names
# We are primarily interested in whether the targets of interest on chormosome 2 were editted; thus the dataframe is filtered to only include alignments from chromosome 2
# The edit_sites data frame creates a new data frame that only contains rows from the chromosome 2 data set in which the "POS" or position matches the values targetted by the CRISPR experiement
# isin(Edit_sites) determines whether each value in the 'POS' column is present in the Edit_sites list
# The output is filered for values that are "True" boolean values from the Edit_stites_df ( i.e positions that are present in both data sets)

In [ ]:
#(2) A data frame with off-target mutations.
off_target_df = df[df['POS'].isin(Edit_sites)== False]
off_target_df

# this data frame represents any mutation (i.e "POS") that is not including in our editting target dataframe

In [ ]:
#(3)- A Summary data frame with number of mutations and genotype per chromosmome considering only off-target changes.

off_target_counts = off_target_df.groupby('CHROM')['GENOTYPE'].value_counts()
off_target_counts

# This data frame groups the values (from our off_target data frame) by chromosome number and genotype
# .value_counts() counts the number of each value in the 'GENOTYPE' column

In [ ]:
#(4) A simple plot of: Value counts of off_target variants between our CRIPSR data and reference mouse genome
import matplotlib.pyplot as plt
off_target_counts.plot(kind='bar', stacked=True)